# 8 数据规整：连接、联合与重塑
重点关注数据联合、连接以及重排

## 8.1 分层索引

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 双层索引的Series
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.421656
   2   -0.125544
   3   -0.201874
b  1    0.484224
   3   -0.674129
c  1    0.466865
   2   -1.578992
d  2   -0.686441
   3   -0.369766
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
# 使用分层索引可以简洁地选择出数据的子集
data['b']

1    0.484224
3   -0.674129
dtype: float64

In [5]:
data['b':'c']

b  1    0.484224
   3   -0.674129
c  1    0.466865
   2   -1.578992
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1    0.484224
   3   -0.674129
d  2   -0.686441
   3   -0.369766
dtype: float64

In [8]:
# 内部层级中也可以选择，比如选择内层中索引为2的
data.loc[:, 2]

a   -0.125544
c   -1.578992
d   -0.686441
dtype: float64

In [11]:
# 重塑数据
data.unstack()

,1,2,3
a,-0.421656,-0.125544,-0.201874
b,0.484224,NaN,-0.674129
c,0.466865,-1.578992,NaN
d,NaN,-0.686441,-0.369766


In [15]:
# 在DataFrame中，每个轴都可以有分层索引
df = pd.DataFrame(np.arange(12).reshape(4, 3),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'California'], ['Green', 'Red', 'Green']])
df.index.names = ['key1', 'key2']
df.columns.names = ['State', 'Color']
df

State      Ohio     California
Color     Green Red      Green
key1 key2                     
a    1        0   1          2
     2        3   4          5
b    1        6   7          8
     2        9  10         11

In [16]:
# 筛选出数据
df['Ohio']

Color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [18]:
df.loc['a']

State  Ohio     California
Color Green Red      Green
key2                      
1         0   1          2
2         3   4          5

In [28]:
from pandas import MultiIndex

columns = MultiIndex.from_arrays([['Ohio', 'Ohio', 'California'], ['Green', 'Red', 'Green']],
                                 names=['State', 'Color'])
indexes = MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                                 names=['key1', 'key2'])

In [29]:
df2 = pd.DataFrame(np.arange(12).reshape(4, 3),
                   columns=columns,
                   index=indexes)

In [30]:
df2

State      Ohio     California
Color     Green Red      Green
key1 key2                     
a    1        0   1          2
     2        3   4          5
b    1        6   7          8
     2        9  10         11

### 8.1.1 重排序和层级排序

In [32]:
# 重新排列轴上的层级顺序,
# swaplevel接收两个层级序号或层级名称，返回一个层级变更的新对象
df2.swaplevel('key1', 'key2')

State      Ohio     California
Color     Green Red      Green
key2 key1                     
1    a        0   1          2
2    a        3   4          5
1    b        6   7          8
2    b        9  10         11

In [36]:
# level=0表示第一层(key1)，level=1表示第二层(key2)
# 如果索引按照字典顺序从最外层开始排序，那么数据选择性能会更好
df2.sort_index(level=0, ascending=False)

State      Ohio     California
Color     Green Red      Green
key1 key2                     
b    2        9  10         11
     1        6   7          8
a    2        3   4          5
     1        0   1          2

### 8.1.2 按层级进行汇总统计
Series和DataFrame上很多描述性和统计行统计有一个level选项，通过level可以指定要在某个特定的轴上进行聚合

In [38]:
df2.groupby(level='key1').sum()

State  Ohio     California
Color Green Red      Green
key1                      
a         3   5          7
b        15  17         19

In [39]:
df2.groupby(level='Color', axis=1).sum()

Color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 使用DataFrame的列进行索引

In [41]:
df3 = pd.DataFrame({'a': range(7),
                    'b': range(7, 0, -1),
                    'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                    'd': [0, 1, 2, 0, 1, 2, 3]})
df3

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [42]:
df3.set_index(['c', 'd'])

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [43]:
# 默认情况下，设置为index的列会从DataFrame中移除，也可以保留
df3.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [46]:
# reset_index是set_index的反向操作
df4 = df3.set_index(['c', 'd'])
df4.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
